# 01. Phân Tích Khám Phá Dữ Liệu (EDA)

Notebook này tập trung vào việc hiểu dữ liệu bán hàng trước khi đưa vào mô hình.

### Mục tiêu:
1. Kiểm tra chất lượng dữ liệu (Missing, Outliers).
2. Phân tích xu hướng doanh thu theo thời gian.
3. Phân tích sản phẩm bán chạy.
4. Phân tích tỷ lệ hủy đơn hàng.

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

PROJECT_ROOT = Path('../')
RAW_DATA_PATH = PROJECT_ROOT / 'data/raw/online_retail_raw.parquet'
CLEAN_DATA_PATH = PROJECT_ROOT / 'data/processed/sales_clean.parquet'

pd.set_option('display.max_columns', None)
print("Libraries loaded.")

## 1. Kiểm tra dữ liệu thô (Raw Data)

In [ ]:
try:
    df_raw = pd.read_parquet(RAW_DATA_PATH)
    print(f"Raw data shape: {df_raw.shape}")
    display(df_raw.head())
except FileNotFoundError:
    print("Raw data not found.")

### Tỷ lệ dữ liệu thiếu (Missing Values)

In [ ]:
missing = df_raw.isnull().sum()
missing = missing[missing > 0]
missing_pct = (missing / len(df_raw)) * 100
pd.DataFrame({'Missing Count': missing, 'Percentage': missing_pct})

### Phân tích đơn hàng bị hủy (Cancelled Orders)
Đơn hàng bị hủy thường có mã bắt đầu bằng 'C'.

In [ ]:
df_raw['InvoiceNo'] = df_raw['InvoiceNo'].astype(str)
cancelled = df_raw[df_raw['InvoiceNo'].str.startswith('C')]
print(f"Số lượng đơn bị hủy: {len(cancelled)} ({len(cancelled)/len(df_raw):.2%})")

# Giá trị hủy
cancelled['TotalValue'] = cancelled['Quantity'] * cancelled['UnitPrice']
print(f"Tổng giá trị bị hủy: {cancelled['TotalValue'].sum():,.2f}")

## 2. Phân tích Dữ Liệu Sạch (Clean Data)
Dữ liệu này đã loại bỏ missing và hủy đơn, dùng cho phân tích chính.

In [ ]:
df = pd.read_parquet(CLEAN_DATA_PATH)
print(f"Clean data shape: {df.shape}")

### Xu hướng doanh thu theo tháng

In [ ]:
df['Month'] = df['InvoiceDate'].dt.to_period('M').astype(str)
monthly_sales = df.groupby('Month')['TotalValue'].sum().reset_index()

fig = px.bar(monthly_sales, x='Month', y='TotalValue', title='Doanh Thu Theo Tháng')
fig.show()

### Biểu đồ nhiệt: Giờ mua hàng vs Các ngày trong tuần
Khách hàng thường mua vào lúc nào?

In [ ]:
df['Hour'] = df['InvoiceDate'].dt.hour
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()

pivot = df.pivot_table(index='DayOfWeek', columns='Hour', values='InvoiceNo', aggfunc='nunique')
# Order days
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Sunday'] # Saturday store closed usually
pivot = pivot.reindex(days)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot, cmap='YlGnBu', annot=False)
plt.title('Tần suất Đơn Hàng: Giờ vs Thứ')
plt.show()

### Top 10 Quốc Gia (Ngoài UK)

In [ ]:
country_sales = df[df['Country'] != 'United Kingdom'].groupby('Country')['TotalValue'].sum().nlargest(10).reset_index()
fig = px.bar(country_sales, x='TotalValue', y='Country', orientation='h', title='Top 10 Thị Trường Quốc Tế')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()